In [ ]:
!mkdir ../checkpoints
!wget https://download.openmmlab.com/mmsegmentation/v0.5/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth -P ../checkpoints

In [2]:
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
import numpy as np

/root/.conda/envs/openmmlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_file = '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/local_configs/segnext/tiny/segnext.tiny.freqfusion.512x512.ade.160k.py'
checkpoint_file = '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/work_dirs/segnext.tiny.freqfusion.512x512.ade.160k/best_mIoU_iter_160000.pth'

In [3]:
# build the model from a config file and a checkpoint file
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

{'type': 'LightHamHeadFreqAware', 'in_channels': [64, 160, 256], 'in_index': [1, 2, 3], 'channels': 256, 'ham_channels': 256, 'dropout_ratio': 0.1, 'num_classes': 150, 'norm_cfg': {'type': 'GN', 'num_groups': 32, 'requires_grad': True}, 'align_corners': False, 'loss_decode': {'type': 'CrossEntropyLoss', 'use_sigmoid': False, 'loss_weight': 1.0}, 'use_checkpoint': False, 'compress_ratio': 4, 'feature_resample_group': 4, 'feature_resample': True, 'comp_feat_upsample': True, 'hamming_window': False, 'semi_conv': True, 'use_high_pass': True, 'use_low_pass': True, 'lowpass_kernel': 5, 'highpass_kernel': 3, 'ham_kwargs': {'MD_R': 16}}
{'in_channels': [64, 160, 256], 'in_index': [1, 2, 3], 'channels': 256, 'dropout_ratio': 0.1, 'num_classes': 150, 'norm_cfg': {'type': 'GN', 'num_groups': 32, 'requires_grad': True}, 'align_corners': False, 'loss_decode': {'type': 'CrossEntropyLoss', 'use_sigmoid': False, 'loss_weight': 1.0}}
spatial True
S 1
D 512
R 16
train_steps 6
eval_steps 7
inv_t 100
eta 

/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(
/root/.conda/envs/openmmlab/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


The model and loaded state dict do not match exactly

unexpected key in source state_dict: backbone.aa2.edge_aware.sigma_min, backbone.aa2.edge_aware.sigma_max, backbone.aa2.edge_aware.sobel_x, backbone.aa2.edge_aware.sobel_y, backbone.aa2.gabor.log_sigma, backbone.aa2.gabor.log_freq, backbone.aa2.gabor.theta, backbone.aa2.gabor.grid, backbone.aa3.edge_aware.sigma_min, backbone.aa3.edge_aware.sigma_max, backbone.aa3.edge_aware.sobel_x, backbone.aa3.edge_aware.sobel_y, backbone.aa3.gabor.log_sigma, backbone.aa3.gabor.log_freq, backbone.aa3.gabor.theta, backbone.aa3.gabor.grid, backbone.aa4.edge_aware.sigma_min, backbone.aa4.edge_aware.sigma_max, backbone.aa4.edge_aware.sobel_x, backbone.aa4.edge_aware.sobel_y, backbone.aa4.gabor.log_sigma, backbone.aa4.gabor.log_freq, backbone.aa4.gabor.theta, backbone.aa4.gabor.grid

missing keys in source state_dict: backbone.aa2.lp.sigma_min, backbone.aa2.lp.sigma_max, backbone.aa2.lp.sobel_x, backbone.aa2.lp.sobel_y, backbone.aa2.gb.log_sigma, backb

In [10]:
# test a single image
model.eval()
img = '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/demo/ADE_val_00001696.jpg'
result = inference_segmentor(model, img)
print(model.state_dict().keys())
print(np.unique(result))      # 原图尺寸
print(result[0].shape)

odict_keys(['backbone.patch_embed1.proj.0.weight', 'backbone.patch_embed1.proj.0.bias', 'backbone.patch_embed1.proj.1.weight', 'backbone.patch_embed1.proj.1.bias', 'backbone.patch_embed1.proj.1.running_mean', 'backbone.patch_embed1.proj.1.running_var', 'backbone.patch_embed1.proj.1.num_batches_tracked', 'backbone.patch_embed1.proj.3.weight', 'backbone.patch_embed1.proj.3.bias', 'backbone.patch_embed1.proj.4.weight', 'backbone.patch_embed1.proj.4.bias', 'backbone.patch_embed1.proj.4.running_mean', 'backbone.patch_embed1.proj.4.running_var', 'backbone.patch_embed1.proj.4.num_batches_tracked', 'backbone.block1.0.layer_scale_1', 'backbone.block1.0.layer_scale_2', 'backbone.block1.0.norm1.weight', 'backbone.block1.0.norm1.bias', 'backbone.block1.0.norm1.running_mean', 'backbone.block1.0.norm1.running_var', 'backbone.block1.0.norm1.num_batches_tracked', 'backbone.block1.0.attn.proj_1.weight', 'backbone.block1.0.attn.proj_1.bias', 'backbone.block1.0.attn.spatial_gating_unit.conv0.weight', 'ba

In [11]:
# show the results
show_result_pyplot(model, img, result, get_palette('ade20k'),out_file='outputs/seg_result.png')

In [3]:
def get_palette(dataset: str) -> list:
    """根据数据集名称返回调色板，支持多种写法（如 ADE20KDataset）"""
    name = dataset.lower()
    if "ade" in name:
        # ADE20K 调色板（简化前 20 类）
         return [
            [120, 120, 120], [180, 120, 120], [6, 230, 230], [80, 50, 50],
            [4, 200, 3], [120, 120, 80], [140, 140, 140], [204, 5, 255],
            [230, 230, 230], [4, 250, 7], [224, 5, 255], [235, 255, 7],
            [150, 5, 61], [120, 120, 70], [8, 255, 51], [255, 6, 82],
            [143, 255, 140], [204, 255, 4], [255, 51, 7], [204, 70, 3],
            [0, 102, 200], [61, 230, 250], [255, 6, 51], [11, 102, 255],
            [255, 7, 71], [255, 9, 224], [9, 7, 230], [220, 220, 220],
            [255, 9, 92], [112, 9, 255], [8, 255, 214], [7, 255, 224],
            [255, 184, 6], [10, 255, 71], [255, 41, 10], [7, 255, 255],
            [224, 255, 8], [102, 8, 255], [255, 61, 6], [255, 194, 7],
            [255, 122, 8], [0, 255, 20], [255, 8, 41], [255, 5, 153],
            [6, 51, 255], [235, 12, 255], [160, 150, 20], [0, 163, 255],
            [140, 140, 140], [250, 10, 15], [20, 255, 0], [31, 255, 0],
            [255, 31, 0], [255, 224, 0], [153, 255, 0], [0, 0, 255],
            [255, 71, 0], [0, 235, 255], [0, 173, 255], [31, 0, 255],
            [11, 200, 200], [255, 82, 0], [0, 255, 245], [0, 61, 255],
            [0, 255, 112], [0, 255, 133], [255, 0, 0], [255, 163, 0],
            [255, 102, 0], [194, 255, 0], [0, 143, 255], [51, 255, 0],
            [0, 82, 255], [0, 255, 41], [0, 255, 173], [10, 0, 255],
            [173, 255, 0], [0, 255, 153], [255, 92, 0], [255, 0, 255],
            [255, 0, 245], [255, 0, 102], [255, 173, 0], [255, 0, 20],
            [255, 184, 184], [0, 31, 255], [0, 255, 61], [0, 71, 255],
            [255, 0, 204], [0, 255, 194], [0, 255, 82], [0, 10, 255],
            [0, 112, 255], [51, 0, 255], [0, 194, 255], [0, 122, 255],
            [0, 255, 163], [255, 153, 0], [0, 255, 10], [255, 112, 0],
            [143, 255, 0], [82, 0, 255], [163, 255, 0], [255, 235, 0],
            [8, 184, 170], [133, 0, 255], [0, 255, 92], [184, 0, 255],
            [255, 0, 31], [0, 184, 255], [0, 214, 255], [255, 0, 112],
            [92, 255, 0], [0, 224, 255], [112, 224, 255], [70, 184, 160],
            [163, 0, 255], [153, 0, 255], [71, 255, 0], [255, 0, 163],
            [255, 204, 0], [255, 0, 143], [0, 255, 235], [133, 255, 0],
            [255, 0, 235], [245, 0, 255], [255, 0, 122], [255, 245, 0],
            [10, 190, 212], [214, 255, 0], [0, 204, 255], [20, 0, 255],
            [255, 255, 0], [0, 153, 255], [0, 41, 255], [0, 255, 204],
            [41, 0, 255], [41, 255, 0], [173, 0, 255], [0, 245, 255],
            [71, 0, 255], [122, 0, 255], [0, 255, 184], [0, 92, 255],
            [184, 255, 0], [0, 133, 255], [255, 214, 0], [25, 194, 194],
            [102, 255, 0], [92, 0, 255]
        ]
    elif "voc" in name:
        return [[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0], [0, 0, 128],
                [128, 0, 128], [0, 128, 128], [128, 128, 128], [64, 0, 0], [192, 0, 0],
                [64, 128, 0], [192, 128, 0], [64, 0, 128], [192, 0, 128], [64, 128, 128],
                [192, 128, 128], [0, 64, 0], [128, 64, 0], [0, 192, 0], [128, 192, 0],
                [0, 64, 128]]
    elif "city" in name:
        return [[128, 64, 128], [244, 35, 232], [70, 70, 70], [102, 102, 156],
                [190, 153, 153], [153, 153, 153], [250, 170, 30], [220, 220, 0],
                [107, 142, 35], [152, 251, 152], [70, 130, 180], [220, 20, 60],
                [255, 0, 0], [0, 0, 142], [0, 0, 70], [0, 60, 100],
                [0, 80, 100], [0, 0, 230], [119, 11, 32], [0, 0, 0]]
    else:
        raise ValueError(f"Unknown dataset: {dataset}")

In [5]:
import os
import numpy as np
from PIL import Image
import cv2
from mmseg.apis import inference_segmentor, init_segmentor

# ✅ ADE20K调色板
def get_palette(dataset='ade20k'):
    if dataset.lower() == 'ade20k':
        return [
            [120, 120, 120], [180, 120, 120], [6, 230, 230], [80, 50, 50],
            [4, 200, 3], [120, 120, 80], [140, 140, 140], [204, 5, 255],
            [230, 230, 230], [4, 250, 7], [224, 5, 255], [235, 255, 7],
            [150, 5, 61], [120, 120, 70], [8, 255, 51], [255, 6, 82],
            [143, 255, 140], [204, 255, 4], [255, 51, 7], [204, 70, 3],
            [0, 102, 200], [61, 230, 250], [255, 6, 51], [11, 102, 255],
            [255, 7, 71], [255, 9, 224], [9, 7, 230], [220, 220, 220],
            [255, 9, 92], [112, 9, 255], [8, 255, 214], [7, 255, 224],
            [255, 184, 6], [10, 255, 71], [255, 41, 10], [7, 255, 255],
            [224, 255, 8], [102, 8, 255], [255, 61, 6], [255, 194, 7],
            [255, 122, 8], [0, 255, 20], [255, 8, 41], [255, 5, 153],
            [6, 51, 255], [235, 12, 255], [160, 150, 20], [0, 163, 255],
            [140, 140, 140], [250, 10, 15], [20, 255, 0], [31, 255, 0],
            [255, 31, 0], [255, 224, 0], [153, 255, 0], [0, 0, 255],
            [255, 71, 0], [0, 235, 255], [0, 173, 255], [31, 0, 255],
            [11, 200, 200], [255, 82, 0], [0, 255, 245], [0, 61, 255],
            [0, 255, 112], [0, 255, 133], [255, 0, 0], [255, 163, 0],
            [255, 102, 0], [194, 255, 0], [0, 143, 255], [51, 255, 0],
            [0, 82, 255], [0, 255, 41], [0, 255, 173], [10, 0, 255],
            [173, 255, 0], [0, 255, 153], [255, 92, 0], [255, 0, 255],
            [255, 0, 245], [255, 0, 102], [255, 173, 0], [255, 0, 20],
            [255, 184, 184], [0, 31, 255], [0, 255, 61], [0, 71, 255],
            [255, 0, 204], [0, 255, 194], [0, 255, 82], [0, 10, 255],
            [0, 112, 255], [51, 0, 255], [0, 194, 255], [0, 122, 255],
            [0, 255, 163], [255, 153, 0], [0, 255, 10], [255, 112, 0],
            [143, 255, 0], [82, 0, 255], [163, 255, 0], [255, 235, 0],
            [8, 184, 170], [133, 0, 255], [0, 255, 92], [184, 0, 255],
            [255, 0, 31], [0, 184, 255], [0, 214, 255], [255, 0, 112],
            [92, 255, 0], [0, 224, 255], [112, 224, 255], [70, 184, 160],
            [163, 0, 255], [153, 0, 255], [71, 255, 0], [255, 0, 163],
            [255, 204, 0], [255, 0, 143], [0, 255, 235], [133, 255, 0],
            [255, 0, 235], [245, 0, 255], [255, 0, 122], [255, 245, 0],
            [10, 190, 212], [214, 255, 0], [0, 204, 255], [20, 0, 255],
            [255, 255, 0], [0, 153, 255], [0, 41, 255], [0, 255, 204],
            [41, 0, 255], [41, 255, 0], [173, 0, 255], [0, 245, 255],
            [71, 0, 255], [122, 0, 255], [0, 255, 184], [0, 92, 255],
            [184, 255, 0], [0, 133, 255], [255, 214, 0], [25, 194, 194],
            [102, 255, 0], [92, 0, 255]
        ]
    else:
        raise NotImplementedError
palette = np.array(get_palette())

# ✅ 上色
def colorize_mask(mask, palette):
    h, w = mask.shape
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)
    for label, color in enumerate(palette):
        color_mask[mask == label] = color
    return color_mask

# ✅ 叠加
def overlay_mask_on_image(image, mask, palette, alpha=0.5):
    color_mask = colorize_mask(mask, palette)
    if color_mask.shape != image.shape:
        color_mask = cv2.resize(color_mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
    overlay = cv2.addWeighted(image, 1 - alpha, color_mask, alpha, 0)
    return overlay

# ✅ 模型加载
model1 = init_segmentor('/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/local_configs/segnext/tiny/segnext.tiny.freqfusion.512x512.ade.160k.py', '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/work_dirs/segnext.tiny.freqfusion.512x512.ade.160k/freq_best_43.4.pth', device='cuda:0')
model2 = init_segmentor('/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/local_configs/segnext/tiny/segnext.tiny.freqfusion.512x512.ade.160k3.py', '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/demo/segnext_tiny_512x512_ade_160k.pth', device='cuda:0')
model1.eval()
model2.eval()

# ✅ 输入输出路径
img_dir = '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/ADEChallengeData2016/images/validation'
gt_dir = '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/ADEChallengeData2016/annotations/validation'
output_dir = 'outputs/compare_overlay/'
os.makedirs(output_dir, exist_ok=True)

# ✅ 批量预测+保存
for filename in os.listdir(img_dir):
    if not filename.endswith('.jpg'):
        continue

    base = filename.replace('.jpg', '')
    img_path = os.path.join(img_dir, filename)
    gt_path = os.path.join(gt_dir, base + '.png')

    print(f'Processing {filename}...')

    # 原图和 GT
    img = np.array(Image.open(img_path).convert('RGB'))
    gt_mask = np.array(Image.open(gt_path))
    if gt_mask.ndim != 2:
        continue

    # 模型预测
    pred1 = inference_segmentor(model1, img_path)[0].astype(np.uint8)
    pred2 = inference_segmentor(model2, img_path)[0].astype(np.uint8)

    # 叠加预测和GT
    pred1_overlay = overlay_mask_on_image(img, pred1, palette)
    pred2_overlay = overlay_mask_on_image(img, pred2, palette)
    gt_overlay = overlay_mask_on_image(img, gt_mask, palette)

    # 横向拼接
    concat = np.concatenate([pred1_overlay, pred2_overlay, gt_overlay], axis=1)

    # 保存
    save_path = os.path.join(output_dir, base + '_compare_overlay.png')
    cv2.imwrite(save_path, cv2.cvtColor(concat, cv2.COLOR_RGB2BGR))

print("✅ 已完成所有图片的叠加预测图对比！")


{'type': 'LightHamHeadFreqAware', 'in_channels': [64, 160, 256], 'in_index': [1, 2, 3], 'channels': 256, 'ham_channels': 256, 'dropout_ratio': 0.1, 'num_classes': 150, 'norm_cfg': {'type': 'GN', 'num_groups': 32, 'requires_grad': True}, 'align_corners': False, 'loss_decode': {'type': 'CrossEntropyLoss', 'use_sigmoid': False, 'loss_weight': 1.0}, 'use_checkpoint': False, 'compress_ratio': 4, 'feature_resample_group': 4, 'feature_resample': True, 'comp_feat_upsample': True, 'hamming_window': False, 'semi_conv': True, 'use_high_pass': True, 'use_low_pass': True, 'lowpass_kernel': 5, 'highpass_kernel': 3, 'ham_kwargs': {'MD_R': 16}}
{'in_channels': [64, 160, 256], 'in_index': [1, 2, 3], 'channels': 256, 'dropout_ratio': 0.1, 'num_classes': 150, 'norm_cfg': {'type': 'GN', 'num_groups': 32, 'requires_grad': True}, 'align_corners': False, 'loss_decode': {'type': 'CrossEntropyLoss', 'use_sigmoid': False, 'loss_weight': 1.0}}
spatial True
S 1
D 512
R 16
train_steps 6
eval_steps 7
inv_t 100
eta 

In [16]:
from PIL import Image
import numpy as np
import os

# ==================================

def colorize_mask(mask, palette):
    h, w = mask.shape
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)
    for label, color in enumerate(palette):
        color_mask[mask == label] = color
    return color_mask

# 设置路径
gt_dir = '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/ADEChallengeData2016/annotations/validation'  # 替换成你的真实路径
output_dir = 'outputs/gt_mask_color/'
os.makedirs(output_dir, exist_ok=True)

# 加载调色板
palette = np.array(get_palette('ade20k'))

# 批量读取标签图并上色
for filename in os.listdir(gt_dir):
    if filename.endswith('.png'):
        gt_path = os.path.join(gt_dir, filename)
        print(f'Processing {filename} ...')

        # ❗ 不做转换，直接读
        gt_mask = np.array(Image.open(gt_path))  # shape 应为 (H, W)
        
        # 检查有效性
        if gt_mask.ndim != 2:
            print(f"Warning: {filename} is not single-channel mask. Skipping.")
            continue

        # 上色并保存
        colored_mask = colorize_mask(gt_mask, palette)
        out_path = os.path.join(output_dir, filename.replace('.png', '_gt_colored.png'))
        Image.fromarray(colored_mask).save(out_path)

print("✅ 所有 ground-truth 标签图已成功上色并保存。")


Processing ADE_val_00001384.png ...
Processing ADE_val_00000371.png ...
Processing ADE_val_00000698.png ...
Processing ADE_val_00000280.png ...
Processing ADE_val_00001673.png ...
Processing ADE_val_00000497.png ...
Processing ADE_val_00000367.png ...
Processing ADE_val_00001473.png ...
Processing ADE_val_00001478.png ...
Processing ADE_val_00000179.png ...
Processing ADE_val_00000643.png ...
Processing ADE_val_00000174.png ...
Processing ADE_val_00001185.png ...
Processing ADE_val_00000874.png ...
Processing ADE_val_00000295.png ...
Processing ADE_val_00001605.png ...
Processing ADE_val_00000818.png ...
Processing ADE_val_00001974.png ...
Processing ADE_val_00001554.png ...
Processing ADE_val_00000111.png ...
Processing ADE_val_00001060.png ...
Processing ADE_val_00001238.png ...
Processing ADE_val_00000650.png ...
Processing ADE_val_00001878.png ...
Processing ADE_val_00000399.png ...
Processing ADE_val_00001312.png ...
Processing ADE_val_00001435.png ...
Processing ADE_val_00000077.

In [4]:
import os
import numpy as np
from PIL import Image
import cv2
from mmseg.apis import inference_segmentor, init_segmentor

# ========= 调色板加载 ==========

palette = np.array(get_palette('ade20k'))

# ========= 颜色映射函数 ==========
def colorize_mask(mask, palette):
    color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    for label, color in enumerate(palette):
        color_mask[mask == label] = color
    return color_mask

# ========= 模型加载 ==========
config1 = '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/local_configs/segnext/tiny/segnext.tiny.freqfusion.512x512.ade.160k3.py'
ckpt1 = '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/demo/segnext_tiny_512x512_ade_160k.pth'
model1 = init_segmentor(config1, ckpt1, device='cuda:0')
model1.eval()

config2 = '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/local_configs/segnext/tiny/segnext.tiny.freqfusion.512x512.ade.160k.py'
ckpt2 = '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/work_dirs/segnext.tiny.freqfusion.512x512.ade.160k/freq_best_43.4.pth'
model2 = init_segmentor(config2, ckpt2, device='cuda:0')
model2.eval()

# ========= 路径设置 ==========
img_dir = '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/ADEChallengeData2016/images/validation'
gt_dir = '/root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/ADEChallengeData2016/annotations/validation'
output_dir = 'outputs/compare_models/'
os.makedirs(output_dir, exist_ok=True)

# ========= 遍历验证集 ==========
for filename in os.listdir(img_dir):
    if not filename.endswith('.jpg'):
        continue

    base = filename.replace('.jpg', '')
    img_path = os.path.join(img_dir, filename)
    gt_path = os.path.join(gt_dir, base + '.png')

    print(f'Processing {filename}...')

    # === 加载原图和GT ===
    img = np.array(Image.open(img_path).convert('RGB'))
    gt_mask = np.array(Image.open(gt_path))
    if gt_mask.ndim != 2:
        print(f"Warning: {filename} 标签图不是单通道，跳过")
        continue

    # === 两个模型预测 ===
    pred1 = inference_segmentor(model1, img_path)[0].astype(np.uint8)
    pred2 = inference_segmentor(model2, img_path)[0].astype(np.uint8)

    # === 上色 ===
    pred1_color = colorize_mask(pred1, palette)
    pred2_color = colorize_mask(pred2, palette)
    gt_color = colorize_mask(gt_mask, palette)

    # === 拼接可视化（横图）===
    concat = np.concatenate([
        cv2.resize(img, (pred1_color.shape[1], pred1_color.shape[0])),
        pred1_color,
        pred2_color,
        gt_color
    ], axis=1)

    # === 保存结果图像 ===
    save_path = os.path.join(output_dir, base + '_compare.png')
    Image.fromarray(concat).save(save_path)

print("✅ 所有图像处理完成，已保存到:", output_dir)


{'type': 'LightHamHead', 'in_channels': [64, 160, 256], 'in_index': [1, 2, 3], 'channels': 256, 'ham_channels': 256, 'dropout_ratio': 0.1, 'num_classes': 150, 'norm_cfg': {'type': 'GN', 'num_groups': 32, 'requires_grad': True}, 'align_corners': False, 'loss_decode': {'type': 'CrossEntropyLoss', 'use_sigmoid': False, 'loss_weight': 1.0}, 'ham_kwargs': {'MD_R': 16}}
{'in_channels': [64, 160, 256], 'in_index': [1, 2, 3], 'channels': 256, 'dropout_ratio': 0.1, 'num_classes': 150, 'norm_cfg': {'type': 'GN', 'num_groups': 32, 'requires_grad': True}, 'align_corners': False, 'loss_decode': {'type': 'CrossEntropyLoss', 'use_sigmoid': False, 'loss_weight': 1.0}}
spatial True
S 1
D 512
R 16
train_steps 6
eval_steps 7
inv_t 100
eta 0.9
rand_init True
load checkpoint from local path: /root/shared-nvme/ac/FreqFusion-main/FreqFusion-main/SegNeXt/demo/segnext_tiny_512x512_ade_160k.pth
The model and loaded state dict do not match exactly

missing keys in source state_dict: backbone.aa2.lp.sigma_min, ba

KeyboardInterrupt: 

In [ ]:
from skimage.morphology import dilation, disk
import cv2, numpy as np, matplotlib.pyplot as plt

def vis_pe_on_boundary(img_t, label_np, save=None):
    """
    img_t     : (3,H,W)  tensor 0-1
    label_np  : (H,W)    numpy int32  segmentation GT
    """
    # ① 计算相位误差能量
    dphi, _ = phase_error(img_t)              # (H,W), ∈(-π,π]
    pe      = torch.sin(dphi/2).abs()         # ∈[0,1]

    # ② 提取边界
    edge = label_np != cv2.erode(label_np, np.ones((3,3),np.uint8))
    edge = dilation(edge, disk(1))            # 可选：膨胀 1-px
    edge_t = torch.from_numpy(edge.astype(np.float32))

    # ③ 只保留边界上的 PE
    pe_edge = pe * edge_t

    # ④ 可视化
    plt.figure(figsize=(5,5))
    plt.imshow(img_t.permute(1,2,0).cpu())       # 先放原图
    plt.imshow(pe_edge.cpu(), cmap='jet', alpha=0.7)
    plt.title('Phase-Error on Seg-Boundary')
    plt.axis('off')
    if save: plt.savefig(save, bbox_inches='tight', dpi=200)
    plt.show()


In [ ]:
# rose_alias_energy.py
# -------------------------------------------------------------
import math, argparse, pathlib, numpy as np
import torch, torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.transforms.functional import to_tensor

# ---------- 0. 工具 ----------
def load_tensor(path: str) -> torch.Tensor:
    p = pathlib.Path(path); ext = p.suffix.lower()
    if ext in ('.pt', '.pth'):
        t = torch.load(p, map_location='cpu')
        if isinstance(t, dict):                # 可能是 ckpt
            t = list(t.values())[0]
    elif ext == '.npy':
        t = torch.from_numpy(np.load(p))
    elif ext in ('.png', '.jpg', '.jpeg', '.bmp'):
        t = to_tensor(Image.open(p).convert('RGB'))
    else:
        raise ValueError(f'Unknown format: {ext}')
    return t.float()

def to_gray(t: torch.Tensor) -> torch.Tensor:      # (C,H,W) or (H,W)
    if t.dim()==3 and t.size(0)==3:
        r,g,b = t
        gray  = 0.299*r + 0.587*g + 0.114*b
    elif t.dim()==3:
        gray = t.mean(0)
    else:  # already (H,W)
        gray = t
    return gray

def gaussian_blur(g: torch.Tensor, sigma=1.5):
    k = int(math.ceil(sigma*6))|1
    ax = torch.arange(-(k//2), k//2+1)
    ker = torch.exp(-(ax**2)/(2*sigma**2))
    ker = (ker/ker.sum()).to(g)
    ker2 = ker[:,None] * ker[None,:]
    pad  = k//2
    return F.conv2d(g[None,None], ker2[None,None], padding=pad)[0,0]

def hp_residual(img_t: torch.Tensor, sigma=1.5):
    g = to_gray(img_t)
    lp = gaussian_blur(g, sigma)
    return g - lp                                # (H,W)

# ---------- 1. 方向能量直方图 ----------
def rose_energy(residual: torch.Tensor, nbins=36):
    H,W = residual.shape
    # 频谱能量
    Fspec = torch.fft.fftshift(torch.fft.fft2(residual, norm='ortho'))
    mag2  = (Fspec.real**2 + Fspec.imag**2)

    # 极坐标角度
    yy, xx = torch.meshgrid(torch.arange(-H//2,H//2),
                            torch.arange(-W//2,W//2), indexing='ij')
    theta = (torch.atan2(yy.float(), xx.float()) + math.pi)   # 0~2π
    bins  = (theta / (2*math.pi) * nbins).long() % nbins

    hist  = torch.zeros(nbins)
    hist.scatter_add_(0, bins.flatten(), mag2.flatten())
    hist = hist.numpy()
    hist /= hist.sum() + 1e-12            # 归一化到 [0,1]
    return hist

# ---------- 2. 画玫瑰图 ----------
def plot_rose(hist_before, hist_after=None, title='Directional HP energy'):
    nb = len(hist_before)
    theta = np.linspace(0, 2*np.pi, nb+1)
    width = 2*np.pi/nb
    ax = plt.subplot(111, projection='polar')
    ax.bar(theta[:-1], hist_before, width=width, color='r', alpha=.5,
           label='before EA')
    if hist_after is not None:
        ax.bar(theta[:-1], hist_after, width=width, color='b', alpha=.5,
               label='after  EA')
    ax.set_theta_zero_location('N'); ax.set_theta_direction(-1)
    ax.set_title(title); ax.legend(loc='upper right')
    plt.show()

# ---------- 3. CLI ----------
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--before', required=True, help='EA-OFF  feature path')
    parser.add_argument('--after',  default=None, help='EA-ON   feature path')
    parser.add_argument('--sigma',  type=float, default=1.5, help='LP σ')
    args = parser.parse_args()

    bef = load_tensor(args.before)
    res0 = hp_residual(bef, args.sigma)
    hist0= rose_energy(res0)

    if args.after:
        aft = load_tensor(args.after)
        res1 = hp_residual(aft, args.sigma)
        hist1= rose_energy(res1)
        plot_rose(hist0, hist1)
        print('[Before] 每方向能量%:', np.round(hist0*100,2))
        print('[After ] 每方向能量%:', np.round(hist1*100,2))
    else:
        plot_rose(hist0, None)
        print('能量分布%:', np.round(hist0*100,2))

if __name__ == '__main__':
    main()
